# Hyperlocal Air Quality Prediction

### Introduction

The [Environmental Defense Fund (EDF)](https://www.edf.org/airqualitymaps) has partnered with Google Earth Outreach to map air pollution at the **hyperlocal level**. In various cities around the world, mobile air quality sensors were used to gather air pollution data at street level. This new approach allows researchers to collect much more data at a granular level and showed how air poluution varied over very short distances. 

Our project focuses on the data collected in [Houston](https://www.edf.org/airqualitymaps/houston), where low cost mobile sensors were outfitted on city fleet vehicles and Google Street View cars, in addition to gathering data from stationary sensors. For this report, we have focused on only one type of pollution - NO2 emissions.

Based on our review of literature and exploring various datasets, we have decided the scope of our project to explore the use of machine learning models to predict air quality at any location in Houston based on meteorlogical conditions, sources of emissions from various facilities, and traffic.

### Inspiration

Our project is inspired and informed by Varsha Gopalakrishnan's "Hyperlocal Air Quality Prediction using Machine Learning" project, outlined on [Medium](https://towardsdatascience.com/hyperlocal-air-quality-prediction-using-machine-learning-ed3a661b9a71). Gopalakrishnan performed her analysis using Oakland data, and we intend to replicate most aspects of the approach for Houston.

### Data

The level of pollution at any place depends on a number of factors like traffic on major streets; emissions from different facilities like railroads, ports, and industries; and meteorological factors like temperature and precipitation. Our target variable is Air Quality Data obtained from the aforementioned pilot done by EDF. We conducted exploratory data analysis for the air quality dataset in our [Houston air quality data Jupyter Notebook](https://github.com/vjoseph21/air-quality-prediction/blob/main/notebooks/cleaning/air_quality.ipynb).

We classified our potential features into the following three buckets:

#### 1. Traffic Data - [Link to notebook](https://github.com/vjoseph21/air-quality-prediction/blob/main/notebooks/cleaning/traffic_data.ipynb)
We look at the geographical range for which EDF collected the air quality data in Houston. The coordinates for the area around Houston is based on minimum and maximum of latitude and longitude from the EDF data. We chose this area as the bounding box to determine location of all traffic signals / intersections within that boundary.

The Overpass API from Open Street Maps is used to determine the location of all traffic signals (intersections) within a given bounding box. The Overpy library is used to send the request to the API and this call returns the latitude and longitude of all traffic signals. Next, the distance between each traffic intersection and each point in the monitoring data is measured. A traffic score is calculated as the 'Number of traffic intersections within 1,000 ft of each point in the monitoring data."

Going forward, in addition to intersections, we will also extend the analysis to look at the distance of bus stops and highways from the NO2 monitoring points.

#### 2. Emissions Data - [Link to notebook](https://github.com/vjoseph21/air-quality-prediction/blob/main/notebooks/cleaning/facility_data.ipynb)

We use NEI's data for point sources - which provides facility specific pollutant information from across the US. Similar to how we bound based on a boundary box region, we consider only those facilities that fall within the (lat, long) range for which EDF data was collected.

After grouping some of the sources based on their types and range of emission values, we calculate the distance between the location of every pollutant data read (from the EDF data) to each facility. We also calculate the emissions per distance measure for each such row, with the idea being that both the quantity of NO2 emission from the facility, as well as the distance from the facility will affect NO2 concentration at a certain point

Going forward, we will invest time on three fronts:
1. Understanding sources currently marked as 'unknown', since they could add vital information (and variance) to the dataset
2. Carrying out a more robuts outlier treatment of the NO2 distribution
3. Calculating additional aggregated numeric metrics such as number of point sources (per source_group) in a given radius from the monitoring point

#### 3. Meteorological factors - [Link to notebook](https://github.com/vjoseph21/air-quality-prediction/blob/main/notebooks/cleaning/met_data.ipynb)
Following Gopalakrishnan's approach, we use the [Daymet](https://daymet.ornl.gov/web_services) API to collect our meteorological data. Daymet's "daily surface weather and climatological summaries" provide daily data at a 1km grid granularity, extrapolated from less-granular meteorological observations. Through approximately 200 calls to the Daymet API, we gathered the available daily data (maximum temperature, minimum temperature, shortwave radiation, vapor pressure, and precipitation) at latitudes and longitudes corresponding to the locations of our air quality measurements, and produced averages for each metric across the 9 months when EDF was collecting air quality data in Houston.

Going forward, we plan to explore alternate approaches to collapsing the daily data for each air quality measurement site - for example, it's possible that minimums or maximums over the 9-month period in question would be more meaningful than averages.

### Modeling 
We will use this dataset to develop a model to predict air quality in neighbourhoods in Houston where the EDF project did not collect air quality data. We plan to first train the machine learning model to predict NO2 emissions on the same locations as the EDF data and then use the trained model and additional traffic, features, and emissions data to preduct concentrations elsewhere in Houston.

For the baseline model, we have merged all the features mentioned above into one dataset and used linear regression for feature selection. We decided to use a threshold of $\pm90$% to indicate very high correlation between features and dropped them to avoid multicollineairty. We then use the remaining features to run a simple linear regression to guage the siginificance of each feature on NO2 emissions. Our results indicate that minimum temperature, precipitation, and number of intersections are the most important features to predict air quality.

In [40]:
#Import basic python packages for data analysis and plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.lines as mlines
import pylab as plot
import matplotlib
import random
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math
import time

### Import Scipy stats packages
from scipy.stats import pearsonr
from scipy.stats import boxcox


# Import statsmodel packages
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices

# import sklearn packages
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import GridSearchCV, cross_validate, cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet

import re
import os


import warnings
warnings.filterwarnings("ignore")

sns.set(style = 'whitegrid')
sns.set_palette('bright')
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## (A) Dataset prep

In [ ]:
# merge all datasets to create master_df

root = os.path.dirname(os.path.dirname(os.getcwd()))

df_aq = pd.read_csv(root + "/data/cleaned/air_quality_NO2.csv", index_col=0)[['value','latitude', 'longitude']]
df_met = pd.read_csv(root + "/data/cleaned/nO2_met.csv", index_col=0)
df_fac = pd.read_csv(root + "/data/cleaned/no2_fac_data.csv", index_col=0)
# df_fac.drop(df_fac.columns[df_fac.columns.str.contains('_emsdist')], axis=1, inplace=True)
df_traffic = pd.read_csv(root + "/data/cleaned/intersection_final.csv", index_col=0)

df_m1 = df_aq.merge(df_met, on = ['latitude', 'longitude'], how = 'inner')
df_m2 = df_m1.merge(df_fac, on = ['latitude', 'longitude'], how = 'inner')
df_merged = df_m2.merge(df_traffic, on = ['latitude', 'longitude'], how = 'inner')
df_merged.drop(columns = ['latitude', 'longitude'], inplace=True)

X = df_merged.drop("value",1) 
y = df_merged["value"]

## (B) Feature engineering

## (C) Feature selection

### (i) Filter method

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = df_merged.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor["value"])
#Selecting highly correlated features
relevant_features = cor_target[(cor_target>0.2) & (cor_target != 1.0)]  # anything above 0.2 gives nothing
relevant_features

In [ ]:
# removing features with high levels of multicollinearity
selected_features_corr = df_merged[list(relevant_features.index)].corr()

# identifying all the features that have a correlation higher than 0.90 or lower than -0.90 indicating a strong positive or negative correlation

threshold_1 = 0.90
threshold_2 = -0.90

def features_high_corr(df_features_corr):
    columns = np.full((df_features_corr.shape[0],), True, dtype=bool)
    for i in range(df_features_corr.shape[0]):
        for j in range(i+1, df_features_corr.shape[0]):
            if (df_features_corr.iloc[i,j] >= threshold_1) | (df_features_corr.iloc[i,j] <= threshold_2) :
                if columns[j]:
                    columns[j] = False
    selected_columns = df_features_corr.columns[columns]
    return selected_columns

# list of features that are not highly correlated
selected_features = features_high_corr(selected_features_corr)
print("Features in dataset that are not highly correlated: ")
print(selected_features)

Comments:

The filter method might not be the most accurate way to go about feature selection. None of the met variables are a part of the selected features despite lowering the threshold to 0.2 for Pearson corr coeff. Things to try:
1) Modify Pearson corr threshold
2) Try other univariate methods apart from Pearson
3) Build a suitable model using these features
4) Compare with other methods below

### (ii) Embedded methods

#### (a) Lasso

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

reg = LassoCV(cv=cv) #tune this -- when supplied a range of potential alphas, best alpha is 0
reg.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))

In [ ]:
coef = pd.Series(reg.coef_, index = X.columns)

print("Lasso picked " + str(sum(coef != 0)) + 
      " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Model")

#### (b) Random Forest

### (iii) Wrapper methods

#### (a) Recursive Feature Elimination (RFE)

In [ ]:
# determining optimum no of features
nof_list=np.arange(1,len(X.columns))            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

In [ ]:
## getting list of optimum features

cols = list(X.columns)
model = LinearRegression()

#Initializing RFE model
rfe = RFE(model, nof)    

#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)

#Fitting the data to model
model.fit(X_rfe,y)

temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

### To-do:
1) Incorporate CV in RFE \
2) Try with different models \
3) Why do we fit the model on all data in second step?

### (iv) Dimensionality Reduction - PCA

## (D) Modeling - Using Cross Validation and Grid Search

In [ ]:
#NB -- Code to get list of params for an estimator:
#regressor.get_params().keys()

### (1) Random forest

In [ ]:
# creating training set (there is no pre-prepared test set for CV)
X_train = X.iloc[:, :].values
y_train = y.iloc[:].values

# scaling the data
feature_scaler = StandardScaler()
X_train = feature_scaler.fit_transform(X_train)

# defining the regressor
regressor = RandomForestRegressor(random_state=0)

# using grid search -- use more hyperparameters
grid_param = {
    'n_estimators': [100, 300, 500, 800, 1000],
    'criterion': ['mse'],
    'bootstrap': [True, False]
}

gd_sr = GridSearchCV(estimator=regressor,
                     param_grid=grid_param,
                     scoring='neg_mean_squared_error',
                     cv=5,
                     n_jobs=-1)

gd_sr.fit(X_train, y_train)

best_parameters = gd_sr.best_params_
print(best_parameters)

best_result = gd_sr.best_score_
print(best_result)

### (2) XGBoost

In [ ]:
# creating training set (there is no pre-prepared test set for CV)
X_train = X.iloc[:200, :3].values
y_train = y.iloc[:200].values

# scaling the data
feature_scaler = StandardScaler()
X_train = feature_scaler.fit_transform(X_train)

# defining the regressor
regressor = XGBRegressor(random_state=0)

# using grid search -- use more hyperparameters
grid_param = {
        'maxdepth':[8],
        'objective':['multi:softmax'],
        'n_estimators':[600, 900, 1200],
        'gamma':[0, .1, .2],
        'lambda':[.5, 1, 3],
        'alpha':[.5, 1, 2],
        'num_class':[3]
        }

gd_sr = GridSearchCV(estimator=regressor,
                     param_grid=grid_param,
                     scoring='neg_mean_squared_error',
                     cv=2,
                     n_jobs=-1)

gd_sr.fit(X_train, y_train)

best_parameters = gd_sr.best_params_
print(best_parameters)

best_result = gd_sr.best_score_
print(best_result)

### (3) Lasso

In [ ]:
# creating training set (there is no pre-prepared test set for CV)
X_train = X.iloc[:200, :3].values
y_train = y.iloc[:200].values

# scaling the data
feature_scaler = StandardScaler()
X_train = feature_scaler.fit_transform(X_train)

# defining the regressor
regressor = Lasso(random_state=0)

# using grid search -- use more hyperparameters
grid_param = {
        'alpha':[1e-3, 1e-2, 1e-1, 1]
        }

gd_sr = GridSearchCV(estimator=regressor,
                     param_grid=grid_param,
                     scoring='neg_mean_squared_error',
                     cv=2,
                     n_jobs=-1)

gd_sr.fit(X_train, y_train)

best_parameters = gd_sr.best_params_
print(best_parameters)

best_result = gd_sr.best_score_
print(best_result)

### (4) Ridge

In [56]:
# creating training set (there is no pre-prepared test set for CV)
X_train = X.values
y_train = y.values

# scaling the data
feature_scaler = StandardScaler()
X_train = feature_scaler.fit_transform(X_train)

# defining the regressor
regressor = Ridge(random_state=0)

# using grid search -- use more hyperparameters
grid_param = {
        'alpha':[1e-3, 1e-2, 1e-1, 1]
        }

gd_sr = GridSearchCV(estimator=regressor,
                     param_grid=grid_param,
                     scoring='neg_mean_squared_error',
                     cv=5,
                     n_jobs=-1)

gd_sr.fit(X_train, y_train)

best_parameters = gd_sr.best_params_
print(best_parameters)

best_result = gd_sr.best_score_
print(best_result)

{'alpha': 0.001}
-1.943138985224071e-07


## (E) Model evaluation / Error analysis